In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import category_encoders as ce

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Loading Data

In [ ]:
property_dfs = []

dir_path = '/content/drive/MyDrive/property_data'

for path in os.listdir(dir_path):
  try:
    property_dfs.append(pd.read_csv(os.path.join(dir_path, path), low_memory=False))
    print(f'Read file: {path}')
  except:
    print(f'Error reading file: {path}')

property_data_2025 = pd.concat(property_dfs, ignore_index=True)

property_data_2025 = property_data_2025[(property_data_2025['PropertyType'] == 'Residential') & (property_data_2025['PropertySubType'] == 'SingleFamilyResidence')]

Read file: CRMLSSold202511.csv
Read file: CRMLSSold202512.csv
Read file: CRMLSSold202510.csv
Read file: CRMLSSold202509.csv
Read file: CRMLSSold202508.csv
Read file: CRMLSSold202507.csv
Read file: CRMLSSold202505.csv
Read file: CRMLSSold202506.csv
Read file: CRMLSSold202503.csv
Read file: CRMLSSold202504.csv
Read file: CRMLSSold202502.csv
Read file: CRMLSSold202501_filled.csv


In [ ]:
lower_bound_close_price = property_data_2025['ClosePrice'].quantile(.005)
upper_bound_close_price = property_data_2025['ClosePrice'].quantile(.995)

property_data_2025 = property_data_2025[(property_data_2025['ClosePrice'] > lower_bound_close_price) & (property_data_2025['ClosePrice'] < upper_bound_close_price)]

In [ ]:
columns_to_keep = [
    'ClosePrice', 'BuyerAgentAOR', 'ListAgentAOR', 'Flooring', 'ViewYN', 'WaterfrontYN', 'BasementYN',
    'PoolPrivateYN','OriginalListPrice', 'ListingKey', 'LivingArea', 'ListPrice', 'DaysOnMarket',
    'BuyerOfficeName', 'CoListOfficeName', 'ListAgentFullName', 'FireplacesTotal', 'AssociationFeeFrequency',
    'AssociationFee', 'AboveGradeFinishedArea', 'ListingKeyNumeric', 'MLSAreaMajor', 'TaxAnnualAmount',
    'AttachedGarageYN', 'ParkingTotal', 'BuilderName', 'YearBuilt', 'ListingId', 'City', 'TaxYear', 'NewConstructionYN',
    'BuildingAreaTotal', 'BedroomsTotal', 'BelowGradeFinishedArea', 'Stories', 'MainLevelBedrooms',
    'PostalCode', 'LotSizeSquareFeet','BathroomsTotalInteger', 'CloseDate'
]
property_data_2025 = property_data_2025[columns_to_keep]

In [ ]:
initial_rows = len(property_data_2025)

property_data_2025.dropna(subset=['ClosePrice'], inplace=True)
property_data_2025.drop_duplicates(inplace=True)

valid_data_mask = (
    (property_data_2025['ClosePrice'] > 0) &             # Price must be positive
    (property_data_2025['LivingArea'] > 100) &           # Living area should be realistic (>100 sqft)
    (property_data_2025['BedroomsTotal'] > 0) &          # At least 1 bedroom for Residential
    (property_data_2025['BathroomsTotalInteger'] > 0)    # Must have a bathroom
)

property_data_2025 = property_data_2025[valid_data_mask].copy()
rows_removed = initial_rows - len(property_data_2025)

print(f"Cleaning Complete.")
print(f"Rows removed due to logic errors: {rows_removed}")
print(f"Remaining clean records: {len(property_data_2025)}")

Cleaning Complete.
Rows removed due to logic errors: 185
Remaining clean records: 128720


In [ ]:
property_data_2025.head()

,ClosePrice,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,...,NewConstructionYN,BuildingAreaTotal,BedroomsTotal,BelowGradeFinishedArea,Stories,MainLevelBedrooms,PostalCode,LotSizeSquareFeet,BathroomsTotalInteger,CloseDate
0,1250000.0,OrangeCounty,OrangeCounty,"Carpet,Tile",False,NaN,NaN,False,1250000.0,1147233684,...,False,NaN,3.0,NaN,1.0,3.0,92646,5913.0,2.0,2025-11-30
1,2299995.0,Mlslistings,Mlslistings,Carpet,False,NaN,NaN,NaN,NaN,1147228247,...,False,NaN,3.0,NaN,NaN,NaN,95124,18432.0,1.0,2025-11-20
2,810000.0,PacificSouthwest,PacificSouthwest,NaN,False,NaN,NaN,False,799900.0,1147223143,...,NaN,NaN,4.0,NaN,2.0,0.0,92173,5300.0,3.0,2025-11-26
3,925000.0,PacificSouthwest,PacificSouthwest,NaN,False,NaN,NaN,False,925000.0,1147209231,...,NaN,NaN,5.0,NaN,2.0,1.0,92154,5272.0,3.0,2025-11-17
4,1300000.0,NorthSanLuisObispo,NorthSanLuisObispo,NaN,False,NaN,NaN,False,1300000.0,1147200364,...,False,NaN,3.0,NaN,1.0,3.0,93465,10500.0,2.0,2025-11-25


# Handle Missing Values

In [ ]:
#Find Missing Values
missing_values = property_data_2025.isnull().sum()
missing_percent = (missing_values / len(property_data_2025)) * 100
missing_info = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percent
})

display(missing_info[missing_info['Missing Values'] > 0].sort_values(by='Percentage', ascending=False))

,Missing Values,Percentage
FireplacesTotal,128720,100.000000
AboveGradeFinishedArea,128720,100.000000
TaxAnnualAmount,128720,100.000000
TaxYear,128720,100.000000
WaterfrontYN,128656,99.950280
BelowGradeFinishedArea,127840,99.316346
BasementYN,125681,97.639062
BuilderName,122662,95.293661
BuildingAreaTotal,120403,93.538689
CoListOfficeName,96714,75.135177


In [ ]:
# Dropping sparse columns
columns_to_drop = [
    'FireplacesTotal', 'AboveGradeFinishedArea', 'TaxAnnualAmount',
    'TaxYear', 'BelowGradeFinishedArea', 'BuilderName',
]

property_data_2025 = property_data_2025.drop(columns=columns_to_drop)

In [ ]:
property_data_2025

,ClosePrice,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,...,City,NewConstructionYN,BuildingAreaTotal,BedroomsTotal,Stories,MainLevelBedrooms,PostalCode,LotSizeSquareFeet,BathroomsTotalInteger,CloseDate
0,1250000.0,OrangeCounty,OrangeCounty,"Carpet,Tile",False,NaN,NaN,False,1250000.0,1147233684,...,Huntington Beach,False,NaN,3.0,1.0,3.0,92646,5913.0,2.0,2025-11-30
1,2299995.0,Mlslistings,Mlslistings,Carpet,False,NaN,NaN,NaN,NaN,1147228247,...,San Jose,False,NaN,3.0,NaN,NaN,95124,18432.0,1.0,2025-11-20
2,810000.0,PacificSouthwest,PacificSouthwest,NaN,False,NaN,NaN,False,799900.0,1147223143,...,San Ysidro,NaN,NaN,4.0,2.0,0.0,92173,5300.0,3.0,2025-11-26
3,925000.0,PacificSouthwest,PacificSouthwest,NaN,False,NaN,NaN,False,925000.0,1147209231,...,San Diego,NaN,NaN,5.0,2.0,1.0,92154,5272.0,3.0,2025-11-17
4,1300000.0,NorthSanLuisObispo,NorthSanLuisObispo,NaN,False,NaN,NaN,False,1300000.0,1147200364,...,Templeton,False,NaN,3.0,1.0,3.0,93465,10500.0,2.0,2025-11-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260050,8050000.0,OrangeCounty,OrangeCounty,NaN,True,NaN,NaN,False,9995000.0,1040062300,...,Seal Beach,False,NaN,4.0,NaN,1.0,90740,7840.0,5.0,2025-01-16
260054,4000000.0,OrangeCounty,OrangeCounty,Vinyl,True,NaN,NaN,False,5499000.0,1038551758,...,Laguna Beach,False,NaN,4.0,NaN,0.0,92651,3000.0,3.0,2025-01-09
260061,4600000.0,RimOTheWorld,RimOTheWorld,"Carpet,Wood",True,NaN,NaN,False,5950000.0,1038339234,...,Lake Arrowhead,False,NaN,6.0,NaN,1.0,92352,16653.0,7.0,2025-01-16
260089,1071000.0,Mlslistings,Mlslistings,NaN,False,NaN,NaN,NaN,1000000.0,1023270484,...,San Jose,False,NaN,3.0,NaN,NaN,95121,6336.0,2.0,2025-01-28


# =====================================================
### BEGIN - Colin Montie
# =====================================================

# Convert Categorical Variables

In [ ]:
# Binary encodes high-cardinality categorical variables
be = ce.BinaryEncoder(cols=['MLSAreaMajor', 'City', 'BuyerOfficeName', 'ListAgentFullName', 'PostalCode', 'CoListOfficeName', 'BuyerAgentAOR', 'ListAgentAOR'])
property_data_2025 = be.fit_transform(property_data_2025)

# Converts boolean variables to binary
bool_map = {True: 1, False: 0, np.nan: 0}
property_data_2025['ViewYN'] = property_data_2025['ViewYN'].map(bool_map)
property_data_2025['NewConstructionYN'] = property_data_2025['NewConstructionYN'].map(bool_map)
property_data_2025['PoolPrivateYN'] = property_data_2025['PoolPrivateYN'].map(bool_map)
property_data_2025['WaterfrontYN'] = property_data_2025['WaterfrontYN'].map(bool_map)
property_data_2025['BasementYN'] = property_data_2025['BasementYN'].map(bool_map)
property_data_2025['AttachedGarageYN'] = property_data_2025['AttachedGarageYN'].map(bool_map)

# Encodes fee frequency with a number representing the number of times per year the fee is due
fee_freq_map = {'Annually': 1, 'Monthly': 12, 'Quarterly': 4, 'SemiAnnually': 2}
property_data_2025['AssociationFeeFrequency'] = property_data_2025['AssociationFeeFrequency'].map(fee_freq_map)

# Fill NA fee frequency rows with mean
property_data_2025['AssociationFeeFrequency'] = property_data_2025['AssociationFeeFrequency'].fillna(property_data_2025['AssociationFeeFrequency'].mean())

# Creates a new df with a binary column for each flooring type
flooring_df = property_data_2025['Flooring'].str.get_dummies(sep=',').add_prefix('Flooring_')

# Concatinates flooring df to rest of data, dropping flooring column
property_data_2025 = pd.concat([property_data_2025.drop(columns=['Flooring']), flooring_df], axis=1)

In [ ]:
# Confirms that categorical variables have been removed
for col in property_data_2025[list(set(property_data_2025.columns) - set(property_data_2025._get_numeric_data().columns))].columns:
  print(f'{col}: {len(property_data_2025[col].unique())}')

ListingId: 128635
CloseDate: 363


# =====================================================
### END - Colin Montie
# =====================================================

# Create Train/Test Split

# =====================================================
### BEGIN - Leah Besser
# =====================================================

In [ ]:
import pandas as pd
import numpy as np

# Convert CloseDate to datetime
property_data_2025['CloseDate'] = pd.to_datetime(property_data_2025['CloseDate'])

# Sort by date
property_data_2025 = property_data_2025.sort_values(by='CloseDate')

In [ ]:
most_recent_month = property_data_2025['CloseDate'].dt.to_period('M').max()
print("Test Month:", most_recent_month)

# Create training set
train = property_data_2025[
    property_data_2025['CloseDate'].dt.to_period('M') != most_recent_month
].copy()

# Create test set (most recent month)
test = property_data_2025[
    property_data_2025['CloseDate'].dt.to_period('M') == most_recent_month
].copy()

print("Training rows:", train.shape[0])
print("Test rows:", test.shape[0])


Test Month: 2025-12
Training rows: 118391
Test rows: 10329


# Normalize Numerical Variables

In [ ]:
numeric_cols = [
    'OriginalListPrice',
    'ListPrice',
    'DaysOnMarket',
    'LivingArea',
    'BedroomsTotal',
    'BathroomsTotalInteger',
    'ParkingTotal',
    'MainLevelBedrooms',
    'LotSizeSquareFeet',
    'YearBuilt'
]
feature_cols = numeric_cols

In [ ]:
X_train = train[feature_cols]
y_train = train['ClosePrice']

X_test = test[feature_cols]
y_test = test['ClosePrice']


In [ ]:
from sklearn.preprocessing import StandardScaler

# Create scaler
scaler = StandardScaler()

# Copy datasets
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Fit ONLY on training data
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])

# Transform test using same scaler
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

# =====================================================
## END - Leah Besser
# =====================================================

# Save Data

In [ ]:
X_test_scaled.to_csv('property_data_test_x.csv', index=False)
X_train_scaled.to_csv('property_data_train_x.csv', index=False)
y_test.to_csv('property_data_test_y.csv', index=False)
y_train.to_csv('property_data_train_y.csv', index=False)